<a href="https://colab.research.google.com/github/gabrielcerono/GlioblastomaMultiforme/blob/main/Beredsen_survival.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn as sk

In [ ]:
!pip install scipy --upgrade

In [ ]:
!pip install --upgrade pip
!pip uninstall --yes --quiet osqp
!pip install -U scikit-survival

In [ ]:
pip install eli5

In [ ]:
from scipy.stats import rankdata


In [ ]:
dataset = pd.read_excel('/content/Beredsen.xlsx')

In [ ]:
dataset.columns

Index(['age', 'biopsy_debulking', 'KPS', 'adjuvant_treatment', 'SVZ status',
       'survival ', 'censoring', 'Unnamed: 7', 'Variables'],
      dtype='object')

In [ ]:
dataset.drop(columns = ['Unnamed: 7', 'Variables'], inplace = True)

In [ ]:
dataset.head()

,age,biopsy_debulking,KPS,adjuvant_treatment,SVZ status,survival,censoring
0,82,1.0,1.0,1.0,0.0,176.0,1.0
1,67,1.0,1.0,2.0,0.0,362.0,0.0
2,70,1.0,1.0,2.0,0.0,433.0,1.0
3,49,2.0,1.0,1.0,0.0,117.0,1.0
4,78,2.0,1.0,1.0,0.0,326.0,1.0


In [ ]:
dataset = dataset.rename(columns={'censoring': 'death', 'survival ' : 'OS_time'})

In [ ]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [ ]:
imp_mean = IterativeImputer(random_state=0)

In [ ]:
data = imp_mean.fit_transform(dataset)

In [ ]:
data = pd.DataFrame(data=data, columns= dataset.columns,)

In [ ]:
data.corr(method = 'pearson')

,age,biopsy_debulking,KPS,adjuvant_treatment,SVZ status,OS_time,death
age,1.000000,-0.082257,-0.161910,-0.029705,0.061247,-0.362111,0.173820
biopsy_debulking,-0.082257,1.000000,0.148640,-0.013053,-0.192748,0.363858,-0.144109
KPS,-0.161910,0.148640,1.000000,0.041573,-0.105957,0.297927,-0.127053
adjuvant_treatment,-0.029705,-0.013053,0.041573,1.000000,-0.015329,0.054873,-0.038990
SVZ status,0.061247,-0.192748,-0.105957,-0.015329,1.000000,-0.260603,0.110869
OS_time,-0.362111,0.363858,0.297927,0.054873,-0.260603,1.000000,-0.476101
death,0.173820,-0.144109,-0.127053,-0.038990,0.110869,-0.476101,1.000000


## Survival analysis and ranking

In [ ]:
Y = data[['death', 'OS_time']]

In [ ]:
Y['death'] = Y['death'].map({1.0: True, 0: False})

In [ ]:
Y = Y.to_records(index=False)

In [ ]:
Y

In [ ]:
X = data.drop(columns=['death', 'OS_time'])

In [ ]:
from sksurv.ensemble import RandomSurvivalForest
from sklearn.model_selection import train_test_split
from eli5.sklearn import PermutationImportance
import eli5

In [ ]:
rsf = RandomSurvivalForest()

In [ ]:
def permutation(X, y, model, loops):
  rankings = np.zeros(len(X.columns),)
  bordarank = np.zeros(len(X.columns),)
  c_score_total = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)

  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same test set
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state = seed )
    X_train, X_rank, y_train, y_rank = train_test_split(
    X_train, y_train, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, y_train)
  #Let's define the permutation.
    permuter = PermutationImportance(
    estimator = model,
    n_iter = 10)
  #Let's fit the permutator on the Y test, (This permutation only shuffle the Y test.)
    permuter.fit(X_rank, y_rank)
  
    columns = X_test.columns.to_list()
  #The feature importance will give us and n array of how much the Concordance score dropped
    feature_importance = permuter.feature_importances_
  # I need to create a ranking out of this
    rankings = np.add(feature_importance, rankings)
  #This will create a numpy array with each c index loss. Each row is a loop. 
    bordarank = np.vstack((bordarank, feature_importance))
  #Let's compute the Concordance index score
    c_score = model.score(X_test, y_test)
    c_score_total.append(c_score)

  
    
  #Dividing to get the average 
  ranking_terminal = np.true_divide(rankings, loops) 
  c_rank = pd.DataFrame({'features': columns, 'C_loss': ranking_terminal}).sort_values(ascending = False, by =['C_loss'])
  c_mean = np.mean(np.array(c_score_total))
  c_std = np.std(np.array(c_score_total))
  #Let's start working with the borda. First we delete the first row that is just 0s
  bordarank = np.delete(bordarank, 0, 0)
  ranking_the_data = rankdata(bordarank * -1, axis=1)
  rankavg = np.mean(ranking_the_data, axis = 0)
  ranksd = np.std(ranking_the_data, axis = 0)
  #Vamos a ver como queda
  borda = pd.DataFrame({'Features': columns, 'C_Average_Loss': ranking_terminal, 'borda_average': rankavg, 'borda_sd': ranksd}).sort_values(ascending = True, by = ['borda_average'])
  



  return c_rank, c_mean, c_std, borda

In [ ]:
rank, c_mean, c_std, borda = permutation(X, Y, rsf, 100)

In [ ]:
c_mean

0.7766496154102278

In [ ]:
c_std

0.01372879766960945

In [ ]:
borda

,Features,C_Average_Loss,borda_average,borda_sd
3,adjuvant_treatment,0.171855,1.00,0.000000
1,biopsy_debulking,0.025196,2.42,0.635295
0,age,0.019631,2.88,0.738647
4,SVZ status,0.010251,3.93,0.724638
2,KPS,0.004276,4.77,0.420833


First top 2 features

In [ ]:
X_tf = data[['adjuvant_treatment', 'biopsy_debulking']]

In [ ]:
def top2features(X, y, model, loops):
  
  c_score_total = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, y_train, y_test = train_test_split(
    X, Y, test_size=0.33, random_state = seed )
    X_train, X_rank, y_train, y_rank = train_test_split(
    X, Y, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, y_train)
  #Let's compute the scores
    c_score = model.score(X_test, y_test)
    c_score_total.append(c_score)

  
  c_mean_top2 = np.mean(np.array(c_score_total))
  c_std_top2 = np.std(np.array(c_score_total))



  return c_mean_top2, c_std_top2

In [ ]:
c_score_tf = top2features(X_tf, Y, rsf, 100)

In [ ]:
c_score_tf

(0.7650182818916558, 0.011195077761728893)

## Brier Score

In [ ]:
pip install pysurvival

In [ ]:
from pysurvival.utils.metrics import concordance_index
from pysurvival.utils.metrics import integrated_brier_score
from pysurvival.models.survival_forest import RandomSurvivalForestModel

In [ ]:
T = data['OS_time']
E = data['death']

In [ ]:
pyforest = RandomSurvivalForestModel(num_trees=100)

In [ ]:
def Py_scores(X, T, E, model, loops):
  
  c_score_total = []
  ibstotal = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, t_train, t_test, e_train, e_test = train_test_split(
    X, T, E, test_size=0.33, random_state = seed )
    X_train, X_rank, t_train, t_rank, e_train, e_rank = train_test_split(
    X_train, t_train, e_train, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, t_train, e_train, max_depth=5)
    
  #Let's compute the C - scores
  #Let's compute the integrated brier score
    ibs = integrated_brier_score(model, X_test, t_test, e_test)
    ibstotal.append(ibs)
    
  
  
  ibstotal2 = np.mean(np.array(ibstotal))
  ibs_std = np.std(np.array(ibstotal))


  return ibstotal2, ibs_std

In [ ]:
ibs_mean, ibs_std = Py_scores(X, T, E, pyforest, 100)

In [ ]:
ibs_mean

0.12826046916225586

In [ ]:
ibs_std

0.005334775141717203

## Deep Surv

In [ ]:
from pysurvival.models.semi_parametric import NonLinearCoxPHModel


In [ ]:
def Py_scores_deepsurv(X, T, E, loops):
  
  c_score_total = []
  ibstotal = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, t_train, t_test, e_train, e_test = train_test_split(
    X, T, E, test_size=0.33, random_state = seed )
    X_train, X_rank, t_train, t_rank, e_train, e_rank = train_test_split(
    X_train, t_train, e_train, test_size=0.5, random_state = seed)
  #Let's train the NN 
    structure = [ {'activation': 'ReLU', 'num_units': 64}, {'activation': 'ReLU', 'num_units': 32}, ]
    nonlinear_coxph = NonLinearCoxPHModel(structure = structure)
    nonlinear_coxph.fit(X_train, t_train, e_train, num_epochs = 500, dropout = 0.2,)
    c_score_total.append(concordance_index(nonlinear_coxph, X_test, t_test, e_test))
    
  #Let's compute the C - scores
  #Let's compute the integrated brier score
    ibs = integrated_brier_score(nonlinear_coxph, X_test, t_test, e_test)
    ibstotal.append(ibs)
    
  
  
  ibstotal2 = np.mean(np.array(ibstotal))
  ibs_std = np.std(np.array(ibstotal))
  c_score_avg = np.mean(np.array(c_score_total))
  c_score_std = np.std(np.array(c_score_total))

  return ibstotal2, ibs_std, c_score_avg, c_score_std

In [ ]:
ib, ibstd, c_index_avg, c_score_std = Py_scores_deepsurv(X, T, E, 100)

In [ ]:
c_index_avg

0.7312816209251735

In [ ]:
c_score_std

0.03631301715480736

In [ ]:
ib

0.13856814493326333

## Cox Proportional Hazards

In [ ]:
from pysurvival.models.semi_parametric import CoxPHModel


In [ ]:
coxph = CoxPHModel()

In [ ]:
def Py_scores_cox(X, T, E, model, loops):
  
  c_score_total = []
  ibstotal = []
  a= 0
  np.random.seed(42); randseed = np.random.randint(9999, size = 1500)
  for x in range(loops):

    #Let's do this trick with the seeding, so we can have the same array of test sets, that we had in the previous feature selection algorithm. 
    a += 1
    seed = randseed[a]
  #Let's do a splitting 1/3 train, 1/3 rank, 1/3 test
    X_train, X_test, t_train, t_test, e_train, e_test = train_test_split(
    X, T, E, test_size=0.33, random_state = seed )
    X_train, X_rank, t_train, t_rank, e_train, e_rank = train_test_split(
    X_train, t_train, e_train, test_size=0.5, random_state = seed)
  #Let's train the model on the 
    model.fit(X_train, t_train, e_train, lr = 1e-2, max_iter = 400)
    c_score_total.append(concordance_index(model, X_test, t_test, e_test))
    
  #Let's compute the C - scores
  #Let's compute the integrated brier score
    ibs = integrated_brier_score(model, X_test, t_test, e_test)
    ibstotal.append(ibs)
    
  
  
  ibstotal2 = np.mean(np.array(ibstotal))
  ibs_std = np.std(np.array(ibstotal))
  c_index = np.mean(np.array(c_score_total))

  return ibstotal2, ibs_std, c_index

In [ ]:
ib, ibstd, c_index_avg = Py_scores_cox(X, T, E, coxph, 10)

In [ ]:
ib

0.13896793715785521

In [ ]:
c_index_avg

0.7341447476172143